In [1]:
from collections import OrderedDict

import datetime

import math

import numpy as np
import pandas as pd

import os

import psycopg2

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
trend = 'bingoblitz_report'
date_saved = 'bingoblitz_report'


workingDirectory = '/Users/MarkRaymond/Desktop/python'
#dir_data = workingDirectory + '/test/'
#dir_date = workingDirectory + '/sample/'

In [4]:
os.chdir(workingDirectory)
os.getcwd()

'/Users/MarkRaymond/Desktop/python'

In [5]:
redshiftQuery = '''


WITH rev AS (
    SELECT
    pae.auction_id,
    sum(CASE WHEN days_passed <= 7
        THEN usd_num_value END) AS d7_rev,
    count(distinct case when event_name = 'purchase'
      and days_passed <= 7 then pae.auction_id end) as d7_purchasers
    FROM post.app_event pae
    JOIN transaction_fact tt ON pae.auction_id = tt.auction_id
    JOIN impression_fact ii ON tt.auction_id = ii.auction_id
    WHERE event_name = 'purchase'
    AND logging_time >= date_trunc('hour',sysdate + interval '-32 day')
    AND ii.advertiser_store_app_id = 'air.com.buffalo_studios.newflashbingo'
    GROUP BY 1
)
select
bid_timestamp::date as date_utc,
dsp_campaign_id as campaign_id,
count(ii.auction_id) as impressions,
sum(cf.count) as clicks,
count(distinct tt.auction_id) as reactivations,
sum(modified_cost * 0.001)as cost,
sum(d7_rev) as d7_rev,
sum(d7_purchasers) as d7_purchasers
from
impression_fact ii
left join (select auction_id, count(clk_timestamp) from click_fact group by 1) as cf on ii.auction_id = cf.auction_id
left join transaction_fact tt on ii.auction_id = tt.auction_id
left join rev on rev.auction_id =  ii.auction_id
where ii.advertiser_store_app_id = 'air.com.buffalo_studios.newflashbingo'
AND bid_timestamp >= date_trunc('hour',sysdate + interval '-32 day')
AND bid_timestamp < date_trunc('hour',sysdate + interval '0 hours')
AND win_timestamp >= date_trunc('hour',sysdate + interval '-32 day')
AND win_timestamp < date_trunc('hour',sysdate + interval '1 day')
group by 1,2
order by 1 desc
;





'''

In [6]:
def runRedshiftQuery(query):
    try:
        redshiftConnection = psycopg2.connect("dbname = 'rtb'  user='rtb_read' host='rtbcluster-west2.cjgay0xd9jwq.us-west-2.redshift.amazonaws.com' password='RTB_password1' port='5439'   ")
    except:
        print('Cannot connect to Reshift')
        return -1
    else:
        print('Connected Successfully')
        df = pd.read_sql(query, redshiftConnection)
        redshiftConnection.close()
        return df
        

In [7]:
df_redshift = runRedshiftQuery(redshiftQuery)

Connected Successfully


In [8]:
df_redshift

,date_utc,campaign_id,impressions,clicks,reactivations,cost,d7_rev,d7_purchasers
0,2019-08-28,116140,1668,69.0,1,15.247345,NaN,NaN
1,2019-08-28,68903,11,NaN,0,0.497515,NaN,NaN
2,2019-08-28,43267,1,NaN,0,0.061959,NaN,NaN
3,2019-08-28,68910,1,NaN,0,0.008111,NaN,NaN
4,2019-08-28,116144,330,12.0,0,0.823194,NaN,NaN
5,2019-08-28,68861,6,NaN,0,0.090539,NaN,NaN
6,2019-08-28,116158,1212,94.0,1,8.349591,NaN,NaN
7,2019-08-28,116185,49,NaN,0,0.445239,NaN,NaN
8,2019-08-28,68896,33,NaN,0,0.622309,NaN,NaN
9,2019-08-28,43265,6,NaN,0,0.082092,NaN,NaN


In [9]:
AthenaQuery = '''


select distinct
c.id as campaign_id,
asa.campaign_type as campaign_type,
d.name as device,
mc.ad_format
from dsp.account_store_app asa
--join account ac on ac.id = asa.account_id
left join dsp.campaign c on c.account_store_app_id = asa.id
left join dsp.campaign_exchange ce on ce.campaign_id = c.id
left join dsp.exchange ex on ex.id = ce.exchange_id
left join dsp.campaign_creative cc on cc.campaign_id = c.id
join mobspire.creative mc on mc.id = cc.creative_id
left join dsp.campaign_country ct on ct.campaign_id = c.id
left join geo.country dc on dc.id = ct.country_id
left join dsp.campaign_device cd on cd.campaign_id = c.id
left join dsp.device d on d.id = cd.device_id
left join dsp.campaign_device_list cdl on c.id = cdl.campaign_id
left join dsp.device_list dl on dl.id = cdl.device_list_id
--left join dsp.campaign_country ct on ct.campaign_id = c.id
--left join geo.country dc on dc.id = ct.country_id
--left join dsp.campaign_device cd on cd.campaign_id = c.id
--left join dsp.device d on d.id = cd.device_id
--left join dsp.campaign_device_list cdl on c.id = cdl.campaign_id
--left join dsp.device_list dl on dl.id = cdl.device_list_id
where asa.id = 2401
--asa.primary_domain in ('candycrushsodasaga.com','candycrushsaga.com','farmheroessaga.com',
--'petrescuesaga.com','bubblewitch3saga.com','pyramidsolitairesaga.com','candycrushjellysaga.com','candycrushfriendssaga.com')
--asa.primary_domain = 'bubblewitch3saga.com'
--and asa.campaign_type = 'RT'
--and jsonb_extract_path_text(custom, 'sampling_group') is not null
--and jsonb_extract_path_text(custom, 'prediction_model') ilike '%cat%'
--and mc.name ilike '%makeafish%'
--and is_live = 't'
--and (asa.name ilike '%ccs%' or asa.name ilike '%ccss%' or asa.name ilike '%ccjs%')
--and c.name ilike '%Spender 6-10%'
order by 1
;





'''

In [10]:
def runAthenaQuery(query):
    try:
        athenaConnection = psycopg2.connect("dbname = 'athena' user='read' host='192.168.19.20' password='sER866NDiPitY'   ")
    except:
        print('Cannot connect to Athena')
        return -1
    else:
        print('Connected Successfully')
        df = pd.read_sql(query, athenaConnection)
        athenaConnection.close()
        return df

In [11]:
df_athena = runAthenaQuery(AthenaQuery)

Connected Successfully


In [12]:
df_athena 

,campaign_id,campaign_type,device,ad_format
0,89130,RT,Android,interstitial
1,89131,RT,Android,interstitial
2,89132,RT,Android,vast
3,89133,RT,Android,vast
4,89134,RT,Android,vast
5,89135,RT,Android,vast
6,89136,RT,Android,native
7,89137,RT,Android,native
8,89138,RT,Android,native
9,89139,RT,Android,native


In [13]:
df_joined = df_redshift.merge(df_athena, on='campaign_id', indicator=True)

In [14]:
df_joined

,date_utc,campaign_id,impressions,clicks,reactivations,cost,d7_rev,d7_purchasers,campaign_type,device,ad_format,_merge
0,2019-08-09,89920,4,NaN,0,0.005495,NaN,NaN,RT,Android,interstitial,both
1,2019-08-08,89920,1317,91.0,17,7.858189,NaN,NaN,RT,Android,interstitial,both
2,2019-08-07,89920,6497,523.0,129,40.443211,16.765,2.0,RT,Android,interstitial,both
3,2019-08-06,89920,4645,344.0,93,24.491309,NaN,NaN,RT,Android,interstitial,both
4,2019-08-05,89920,5547,428.0,92,30.951589,NaN,NaN,RT,Android,interstitial,both
5,2019-08-04,89920,4469,310.0,91,23.198278,NaN,NaN,RT,Android,interstitial,both
6,2019-08-03,89920,4480,322.0,93,25.171437,NaN,0.0,RT,Android,interstitial,both
7,2019-08-02,89920,4396,321.0,94,24.107679,NaN,NaN,RT,Android,interstitial,both
8,2019-08-01,89920,4463,300.0,74,23.435913,NaN,NaN,RT,Android,interstitial,both
9,2019-07-31,89920,4508,383.0,96,25.841984,4.193,1.0,RT,Android,interstitial,both


In [15]:
sum_fields = [ 'impressions', 'clicks', 'reactivations', 'cost',
                  'd7_rev', 'd7_purchasers']

def get_aggs():
    sum_fields = [ 'impressions', 'clicks', 'reactivations', 'cost',
                  'd7_rev', 'd7_purchasers', ]
    sum_dict = OrderedDict([(field, 'sum') for field in sum_fields])

   # mean_fields = ['cpi', 'd7_roi', 'avg_pred_prob', 'avg_bid_price',
   #                'avg_win_price', 'clk_imp_ratio', 'txn_imp_ratio',
   #                'txn_clk_ratio', 'cpp_imp_ratio', 'cppr_imp_ratio',
   #                'cpp_txn_ratio', 'cppr_txn_ratio']
   # mean_dict = OrderedDict([(field, 'mean') for field in mean_fields])

    aggs = OrderedDict(sum_dict.items()) # + mean_dict.items())   
    return aggs

In [16]:
group_idx = ['date_utc', 'campaign_type', 'device', 'ad_format']
aggs = get_aggs()

In [17]:
df_agg = df_joined.groupby(group_idx).agg(aggs)

In [18]:
df_agg

impressions  clicks  \
date_utc   campaign_type device  ad_format                           
2019-07-27 RT            Android banner              21787   105.0   
                                 interstitial        10048   843.0   
                                 native               5370   111.0   
                                 vast                16948  3324.0   
2019-07-28 RT            Android banner              41677   209.0   
                                 interstitial        16414  1305.0   
                                 native               7757   167.0   
                                 vast                29728  6117.0   
2019-07-29 RT            Android banner              40001   223.0   
                                 interstitial        14428  1120.0   
                                 native               8185   128.0   
                                 vast                28917  5762.0   
2019-07-30 RT            Android banner              38810   224.0   
                                 interstitial        16828  1417.0   
                                 native               8228   134.0   
                                 vast                31023  6349.0   
2019-07-31 RT            Android banner              37773   199.0   
                                 interstitial        14898  1305.0   
                                 native               7367   136.0   
                                 vast                29020  5734.0   
2019-08-01 RT            Android banner              36950   195.0   
                                 interstitial        15078  1194.0   
                                 native               7399   134.0   
                                 vast                29154  5524.0   
2019-08-02 RT            Android banner              35552   217.0   
                                 interstitial        14908  1254.0   
                                 native               7611   142.0   
                                 vast                30039  5273.0   
2019-08-03 RT            Android banner              32813   160.0   
                                 interstitial        12429   911.0   
                                 native               7026   121.0   
                                 vast                28441  5338.0   
2019-08-04 RT            Android banner              38478   147.0   
                                 interstitial        13844  1073.0   
                                 native               7188   136.0   
                                 vast                29411  5471.0   
2019-08-05 RT            Android banner              42866   168.0   
                                 interstitial        15896  1256.0   
                                 native               7449   155.0   
                                 vast                30814  5715.0   
2019-08-06 RT            Android banner              39997   176.0   
                                 interstitial        14896  1186.0   
                                 native               7121   134.0   
                                 vast                29949  5387.0   
2019-08-07 RT            Android banner              46499   256.0   
                                 interstitial        18230  1596.0   
                                 native               7762   154.0   
                                 vast                32183  5675.0   
2019-08-08 RT            Android banner              18493   181.0   
                                 interstitial         4712   366.0   
                                 native               2726    60.0   
                                 vast                12900  2514.0   
2019-08-09 RT            Android banner                 80     0.0   
                                 interstitial            7     0.0   
                                 vast                    1     0.0   

                                               reactivat

In [19]:
df_agg.to_csv("bingoblitz_report.csv", encoding='utf-8')

In [20]:
print(datetime.datetime.now())

2019-08-28 15:47:24.507981
